In [1]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd




data=gutenberg.raw('shakespeare-hamlet.txt')


with open('hamlet.txt' , 'w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [5]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


with open('hamlet.txt' , 'r') as file:
    text = file.read().lower()
    
    
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1

In [12]:
input_sequences=[]

for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [13]:
max_sequences_len=max([len(x) for x in input_sequences])
max_sequences_len

14

In [14]:
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequences_len,padding='pre'))

In [15]:
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [16]:
import tensorflow as tf
x,y = input_sequences[:,:-1],input_sequences[:,-1]

In [17]:
x

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]])

In [19]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [20]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout


model=Sequential()
model.add(Embedding(total_words,100,input_length=max_sequences_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation='softmax'))


model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 13, 100)           481800    
                                                                 
 lstm_2 (LSTM)               (None, 13, 150)           150600    
                                                                 
 dropout_1 (Dropout)         (None, 13, 150)           0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               100400    
                                                                 
 dense_1 (Dense)             (None, 4818)              486618    
                                                                 
Total params: 1219418 (4.65 MB)
Trainable params: 1219418 (4.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
history=model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test),verbose=1)

Epoch 1/50


644/644 [==============================] - 11s 14ms/step - loss: 6.9081 - accuracy: 0.0327 - val_loss: 6.7739 - val_accuracy: 0.0356
Epoch 2/50
644/644 [==============================] - 8s 13ms/step - loss: 6.4633 - accuracy: 0.0373 - val_loss: 6.8842 - val_accuracy: 0.0424
Epoch 3/50
644/644 [==============================] - 8s 13ms/step - loss: 6.3307 - accuracy: 0.0440 - val_loss: 6.9194 - val_accuracy: 0.0488
Epoch 4/50
644/644 [==============================] - 8s 13ms/step - loss: 6.1870 - accuracy: 0.0504 - val_loss: 6.9155 - val_accuracy: 0.0513
Epoch 5/50
644/644 [==============================] - 8s 13ms/step - loss: 6.0547 - accuracy: 0.0565 - val_loss: 6.9537 - val_accuracy: 0.0544
Epoch 6/50
644/644 [==============================] - 8s 13ms/step - loss: 5.9137 - accuracy: 0.0612 - val_loss: 7.0258 - val_accuracy: 0.0596
Epoch 7/50
644/644 [==============================] - 8s 13ms/step - loss: 5.7742 - accuracy: 0.0701 - val_loss: 7.0887 - val_accuracy: 0.0

In [24]:
def predict_next_word(model,tokenizer,text,max_sequences_len):
    token_list=tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequences_len:
        token_list = token_list[-(max_sequences_len-1):]
    token_list=pad_sequences([token_list],maxlen=max_sequences_len-1,padding='pre')
    predicted=model.predict(token_list,verbose=0)
    predicted_word_index = np.argmax(predicted,axis=1)
    for word,index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
        
    return None

In [25]:
input_text = "To be or not to be"

print(f'Input text:{input_text}')
max_sequences_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequences_len)
print(f'Next Word Predict:{next_word}')

Input text:To be or not to be
Next Word Predict:considered


In [26]:
model.save('next_word_lstm.h5')

c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [27]:
import pickle
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [33]:
def predict_next_word_from_text(input_text, model, tokenizer):
    print(f'Input text: {input_text}')
    max_sequences_len = model.input_shape[1] + 1
    next_word = predict_next_word(model, tokenizer, input_text, max_sequences_len)
    print(f'Next Word Predict: {next_word}')
    return None

In [36]:
input_text = "Is not this something more then"
predict_next_word_from_text(input_text, model, tokenizer)

Input text: Is not this something more then
Next Word Predict: fantasie
